In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium 
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from glob import glob
import os
pbar = ProgressBar()
pbar.register()

from tqdm import tqdm_notebook as tqdm
import geopandas as gpd

#한글깨짐 방지
plt.rc('font', family = 'Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

In [2]:
s_2017 = pd.read_csv('E:/python/data/신한카드데이터/내국인 유입지별_2017년_신한_종합.csv').iloc[:,1:]
s_2016 = pd.read_csv('E:/python/data/신한카드데이터/내국인 유입지별_2016년_신한_종합.csv').iloc[:,1:]
gu = gpd.read_file('E:/python/data/1. 공통참조파일/1. 지역경계 shp파일/통계청/2016년/서울시(집계구)_EPSG_5179.shp',
                  encoding = 'cp949')

In [218]:
#key값지정
code_book = {}
code_book[1] = "20세이하"
code_book[2] = "21~30세"
code_book[3] = "31~40세"
code_book[4] = "41~50세"
code_book[5] = "51~60세"
code_book[6] = "61세이상"
code_book['t1'] = "00시00~05시59"
code_book['t2'] = "06시00~11시59"
code_book['t3'] = "12시00~17시59"
code_book['t4'] = "18시00~23시59"
code_book['F'] = "여성"
code_book['M'] = "남성"
code_book["1"] = "주말"
code_book["0"] = "주중"
#컬럼명
code_book['USECT_CORR'] = "평균 이용건수"
code_book['AMT_CORR'] = "평균 이용금액"
code_book['C_PER_AMT'] = "건별 평균 이용금액"
code_book['AGE_GB'] = "연령대"
code_book['weekend'] = "주말"
code_book['SEX_CCD'] = "성별"
code_book['TM'] = "시간대"
code_book['SB_UPJONG'] = '업종'

#업종추가
upjong = pd.read_csv('./data/041. 서울시 15-17년 업종별 내외국인 신한카드 매출데이터/2. 파일데이터/신한카드_내국인_업종코드.csv',
                    encoding= 'cp949')
upjong['UPJONG_NM'] = upjong['UPJONG_NM'].apply(lambda x : x.replace('/','&')) 
upjong.index = upjong.UPJONG
del upjong['UPJONG']
upjong_dict = upjong.to_dict()
upjong_dict = upjong_dict['UPJONG_NM']
code_book.update(upjong_dict)

In [5]:
#C_SGG_CD에 결측이 있으나, 이는 오류가 아님 (서울경기 외 다른 지역 시도)
s_2017.shape, s_2017[s_2017.C_SGG_CD.isna()].shape

((3260210, 10), (632094, 10))

In [6]:
def gu_merge(df):
    #집계구 코드 불러오고 발라내기 
    gwang_code = pd.read_csv('./data/광진구_행정동코드.csv',encoding = 'cp949')
    gwangjin_dong = gwang_code['행정동'].values
    gwangjin_gu = gu[gu.ADM_NM.isin(gwangjin_dong)]

    # 각 집계구의 중심점
    gwangjin_gu['center_point'] = gwangjin_gu.geometry.apply(lambda x : x.centroid)
    gwangjin_gu['center_x'] = gwangjin_gu.center_point.apply(lambda point : point.x)
    gwangjin_gu['center_y'] = gwangjin_gu.center_point.apply(lambda point : point.y)
    
    #데이터 결합
    df.BLOCK_ID = df.BLOCK_ID.astype(str) #타입이 다르므로 통일
    merged = gwangjin_gu.merge(df, how = 'outer',left_on = 'TOT_REG_CD',right_on = 'BLOCK_ID')
    
    #결측제외 (집계구 개수와, 데이터의 집계구 개수가 다름)
    merged = merged[~(merged.BLOCK_ID.isna())]
    return merged

In [8]:
s_2017_gu = gu_merge(s_2017)
s_2016_gu = gu_merge(s_2016)

C:\ProgramData\Anaconda3\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  super(GeoDataFrame, self).__setitem__(key, value)
C:\ProgramData\Anaconda3\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  super(GeoDataFrame, self).__setitem__(key, value)
C:\ProgramData\Anaconda3\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [9]:
def near_park(df):
    '''범위내 축소 
    
    '''
    # 대공원주변 범위
    ymin,ymax = 1949300, 1951100
    xmin,xmax = 962000, 964100
    
    target = df[(df.center_x >= xmin)&(df.center_x <= xmax)]
    target = target[(target.center_y >= ymin)&(target.center_y <= ymax)]
    
    #건대상권 제외
    gun_x, gun_y = 962250, 1949600
    target = target[~((target.center_x <= gun_x)&((target.center_y <= gun_y)))]
    
    return target

In [103]:
near_2016 = near_park(s_2016_gu)
near_2017 = near_park(s_2017_gu)

In [104]:
def labeling(df):
    '''BLOCK_ID값으로 label매칭
    '''
    label = pd.read_csv('./data/신한카드데이터/블럭별 클러스터라벨.csv')
    label['BLOCK_ID'] = label.BLOCK_ID.astype(str)
    labeled = pd.merge(df, label,how= 'left',on = 'BLOCK_ID')
    return labeled

In [157]:
near_2016_lb = labeling(near_2016)
near_2017_lb = labeling(near_2017)

In [158]:
def add_weekend(df):
    #주말구분
    df.loc[(df.DAW_CCD == 1)|(df.DAW_CCD == 7),'weekend'] = "1"
    df.loc[~((df.DAW_CCD == 1)|(df.DAW_CCD == 7)),'weekend'] = "0"
    return df

In [159]:
near_2016_lb = add_weekend(near_2016_lb)
near_2017_lb = add_weekend(near_2017_lb)

In [160]:
#결측값을 임의의 숫자로 체워둠
near_2016_lb= near_2016_lb.fillna('0')
near_2017_lb= near_2017_lb.fillna('0')

In [161]:
### 좌표 데이터 기억
geo_2017 = near_2016_lb[['BLOCK_ID','geometry']]
geo_2016 = near_2017_lb[['BLOCK_ID','geometry']]

geo_2017 = geo_2017.drop_duplicates()
geo_2016 = geo_2016.drop_duplicates()

geo_m = pd.concat([geo_2017,geo_2016])
geo_m = geo_m.drop_duplicates().reset_index(drop = True)

In [162]:
def agg_cluster(df):
    #11개 구역 클러스터링 한 내역
    label_line = gpd.read_file('E:/python/data/신한카드데이터/라벨별_명칭_geo값.shp')
    
    cluster_df = df[['SB_UPJONG','TS_YM','TM','labels','C_SIDO_CD','C_SGG_CD','weekend','USECT_CORR','AMT_CORR']]
    cluster_g = cluster_df.groupby(['SB_UPJONG','TS_YM','TM','C_SIDO_CD','C_SGG_CD','labels','weekend'])['USECT_CORR','AMT_CORR'].sum().reset_index()
    cluster_g = cluster_g.merge(label_line, how= 'left')
    cluster_g = gpd.GeoDataFrame(cluster_g)
    
    return cluster_g

In [163]:
c_2016 = agg_cluster(near_2016_lb)
c_2017 = agg_cluster(near_2017_lb)

###  유입지코드

In [186]:
def encoding_name(df):
    '''
    유입지코드 불러와서 매칭시키기
    '''
    comefrom = pd.read_csv('E:/python/data/041. 서울시 15-17년 업종별 내외국인 신한카드 매출데이터/2. 파일데이터/신한카드_내국인_유입지코드.csv',
                      encoding = 'cp949')
    comefrom =comefrom.fillna('0')
    #딕셔너리화 
    comefrom['CODE'] = comefrom['C_SIDO_CD'].astype(str)+"_"+comefrom['C_SGG_CD'].astype(int).astype(str)
    comefrom['NAME'] = comefrom['C_SIDO_NM']+"_"+comefrom['C_SGG_NM']
    C_SIDO_dict = comefrom[['CODE','NAME']]
    C_SIDO_dict = C_SIDO_dict.set_index('CODE')
    C_SIDO_dict = C_SIDO_dict.to_dict()['NAME']

    
    #매칭
    df['CODE'] = df.C_SIDO_CD.astype(int).astype(str) +'_'+df.C_SGG_CD.astype(int).astype(str)
    df['C_NAME'] = df.CODE.apply(lambda x : C_SIDO_dict[x])
    df['C_NAME'] = df['C_NAME'].apply(lambda x : x.replace('_0',''))
    return df

In [187]:
c_2016_c = encoding_name(c_2016)
c_2017_c = encoding_name(c_2017)

In [259]:
def comefrom_rate_b(df):
    #저장위치
    path = './output/신한카드/유입지비율_월평균'
    os.makedirs(path,exist_ok = True)
    
    year = str(df['TS_YM'][0])[:4]
    
    if year == 2016:
        divid = 12
    else:
        divid = 11
    
    sample = (df.groupby(['names','C_NAME'])['USECT_CORR','AMT_CORR'].sum()/divid).reset_index()
    #건별 평균 이용금액
    sample['C_PER_AMT'] = round(sample['AMT_CORR']/sample['USECT_CORR'],2)
    base_top10 = pd.DataFrame()
    
    for c in ['AMT_CORR','USECT_CORR','C_PER_AMT']:
        for name in sample.names.unique():
            sample_part = sample[sample.names == name]
            sample_part = sample_part.set_index('C_NAME')[c]
            sample_part = round((sample_part/sample_part.sum())*100,2).sort_values(ascending= False)
            sample_part.to_csv(f'{path}/{year}_{name}_{code_book[c]}에 따른 유입지비율.csv',encoding = 'cp949')
            
            #상위 10개조합
            top10 = pd.DataFrame(sample_part.head(10)).reset_index()
            top10.columns = [name, name]
            base_top10 = pd.concat([base_top10,top10],axis=1)
        base_top10.to_csv(f'{path}/{year}_{code_book[c]}에 따른 유입지top10.csv',encoding = 'cp949')
        #리셋
        base_top10 = pd.DataFrame()
        
    
    return base_top10
        

In [260]:
base_top10_2016 = comefrom_rate_b(c_2016_c)
base_top10_2017 = comefrom_rate_b(c_2017_c)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [361]:
def get_top5(df,value_col):
    #저장위치
    year = str(df['TS_YM'].iloc[0])[:4]
    
    if year == 2016:
        divid = 12
    else:
        divid = 11
    
    sample = (df.groupby(['names','C_NAME'])['USECT_CORR','AMT_CORR'].sum()/divid).reset_index()
    #건별 평균 이용금액
    sample['C_PER_AMT'] = round(sample['AMT_CORR']/sample['USECT_CORR'],2)
    
    top_5_dict = {}
    for name in sample.names.unique():
        sample_part = sample[sample.names == name]
        sample_part = sample_part.set_index('C_NAME')[value_col]
        sample_part = round((sample_part/sample_part.sum())*100,2).sort_values(ascending= False)
        top_5_dict[name] = sample_part.head(5).index.values
    
    return top_5_dict

In [302]:
top5_2016 = get_top5(c_2016_c, "AMT_CORR")

### top5의 업종

In [321]:
col = 'names'
hue = 'SB_UPJONG'
value_col = 'AMT_CORR'
name = '구의문'
color_bound = None
cmap = 'tab20'
special_legend = None

In [333]:
# s_upjong1 = ['한식','일식&중식&양식','제과점','커피전문점',
#            '패스트푸드','기타요식']
# s_upjong2 = ['유흥','음&식료품','의류&잡화','스포츠&문화&레저',
#             '교육용품','가전가구','자동차']
s_upjong1 = ['sb01','sb02','sb03','sb04','sb05','sb06']
s_upjong2 = ['sb07','sb09','sb10','sb11','sb18','sb20','sb21'] #sb17 유아교육을 제외하고도 실험
s_upjong_m = s_upjong1 + s_upjong2

In [362]:
def top5_upjong(df, hue, value_col, color_bound = None,special_legend = None,cmap = 'tab20',part = None):
    
    path = './output/신한카드/유입지_상위_5개지역_업종'
    os.makedirs(path,exist_ok = True)
    year = str(df['TS_YM'].iloc[0])[:4]
    
    if year == 2016:
        divid = 12
    else:
        divid = 11
    #특정값만
    if part != None:
        df = df[df[hue].isin(part)]
        
    grouped = (df.groupby(['names','C_NAME',hue])['USECT_CORR','AMT_CORR'].sum()/12).reset_index()
    grouped['C_PER_AMT'] = grouped['AMT_CORR']/grouped['USECT_CORR']
    
    #top5
    top5_dict= get_top5(df, value_col)
    
    for name in grouped.names.unique():
        c_grouped = grouped[grouped.names == name] #특정클러스터
        top5 =top5_dict[name]
        c_grouped = c_grouped[c_grouped.C_NAME.isin(top5)]
        grouped_pivot = c_grouped.pivot_table(index= hue ,columns= "C_NAME",values = value_col, 
                                       aggfunc = 'sum',fill_value=0)

        #비율로 변경
        sumed = grouped_pivot.sum(axis= 0).values
        grouped_pivot = (grouped_pivot/sumed)*100

        #값 누적(누적막대그래프를 그리기 위해)
        for i in range(1,len(grouped_pivot)):
            grouped_pivot.values[i] = grouped_pivot.values[i-1]+ grouped_pivot.values[i]

        label_num = grouped_pivot.shape[0] #hue개수

        #사용할 색상
        c_min = 0.0
        c_max = 1.0
        if color_bound != None:
            c_min = color_bound[0]
            c_max = color_bound[1]
        color_list = [plt.cm.get_cmap(cmap)(c) for c in np.linspace(c_min, c_max,label_num)]  #np.linspace 범위내 동일한 간격으로 값 생성

        legends = list(grouped_pivot.index.values) 

        #코드북에 있는 값 가져오기
        for i, l_v in enumerate(legends):
            legends[i] = code_book[l_v]
        legends.reverse()


        #특정범례 지정시
        if special_legend != None:
            legends = special_legend

        #그리기 (누적값이 높은 row순으로 차례로 그려나감 - 겹치는 형태)
        y_value = grouped_pivot.columns #누적을 그릴 컬럼명
        x_value = grouped_pivot.values #누적되어질 값
        plt.figure(figsize=(12,8))
        for i in range(len(grouped_pivot)):
            ax = plt.barh(y_value, x_value[label_num-1-i],color= color_list[i])

        plt.xticks(size = 12)
        plt.yticks(size = 12)
        plt.title(f'{year}_{name}_상위5개 유입지_{code_book[hue]}_{code_book[value_col]}')
        plt.legend(legends,loc = 'center left',bbox_to_anchor=(1, 0.5),fontsize = 12)
        plt.tight_layout()
        plt.savefig(f'{path}/{year}_{name}_상위5개 유입지_{code_book[hue]}_{code_book[value_col]}.png')
        plt.close()

In [349]:
top5_upjong(c_2016_c,"SB_UPJONG","AMT_CORR",part = s_upjong1)
top5_upjong(c_2016_c,"SB_UPJONG","USECT_CORR",part = s_upjong1)
top5_upjong(c_2016_c,"SB_UPJONG","C_PER_AMT",part = s_upjong1)

In [363]:
top5_upjong(c_2016_c,"SB_UPJONG","AMT_CORR",part = s_upjong2)
top5_upjong(c_2016_c,"SB_UPJONG","USECT_CORR",part = s_upjong2)
top5_upjong(c_2016_c,"SB_UPJONG","C_PER_AMT",part = s_upjong2)